In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time  # Import time module for sleep function

# List of years you want to fetch data for
years = list(range(2016, 2024))  # 2013 to 2023 inclusive

# Initialize an empty list to hold DataFrames for each year
dataframes = []

# Loop over each year, fetch and parse the data
for year in years:
    url = f'https://www.pro-football-reference.com/years/{year}/kicking.htm'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'id': 'kicking'})

        # Assuming the table structure is consistent across the years
        header_row = table.find('thead').find_all('tr')[-1]
        headers = [th.get_text() for th in header_row.find_all('th')]

        # Adjust headers to exclude 'Rk' if present
        if headers[0] == 'Rk':
            headers = headers[1:]  # Adjust according to the structure observed

        # Collect row data
        rows = []
        for row in table.find('tbody').find_all('tr'):
            cols = row.find_all('td')
            if cols:  # Ensure it's not a header row or empty row
                row_data = [col.get_text() for col in cols]
                rows.append(row_data)

        # Create a DataFrame for the current year and add a year column
        df = pd.DataFrame(rows, columns=headers)
        df['Year'] = year  # Add a year column to keep track of the data year
        dataframes.append(df)
    else:
        print(f"Failed to fetch data for year {year}")

    # Sleep for 3 seconds to limit to 20 requests per minute
    time.sleep(3)

# Concatenate all the DataFrames into a single DataFrame
kicking_data = pd.concat(dataframes, ignore_index=True)

# Clean up the data as specified
kicking_data = kicking_data.drop(['KO','KOYds','TB','TB%','KOAvg'], axis=1)
kicking_data = kicking_data.fillna(" ")
kicking_data = kicking_data.dropna(subset=['FGA', 'FGM', 'FGA','FGM', 'FGA','FGM', 'FGA','FGM', 'FGA','FGM', 'FGA','FGM','Lng','FG%', 'XPA', 'XPM', 'XP%'], how='all')

# Print or process the DataFrame as needed
print(kicking_data.head())


            Player   Tm Age Pos   G GS FGA FGM FGA FGM  ... FGA FGM FGA FGM  \
0   Dustin Hopkins  WAS  26   K  16  0          12  12  ...   7   3  42  34   
1    Caleb Sturgis  PHI  27   K  16  0          12  12  ...   6   4  41  35   
2       Nick Novak  HOU  35   K  16  0   1   1  12  12  ...   6   3  41  35   
3  Justin Tucker*+  BAL  27   K  16  0           4   4  ...  10  10  39  38   
4      Graham Gano  CAR  29   K  16  0           4   4  ...   6   3  38  30   

  Lng    FG% XPA XPM     XP%  Year  
0  53  81.0%  39  36   92.3%  2016  
1  55  85.4%  31  30   96.8%  2016  
2  53  85.4%  25  22   88.0%  2016  
3  57  97.4%  27  27  100.0%  2016  
4  54  78.9%  34  31   91.2%  2016  

[5 rows x 24 columns]


In [73]:
import numpy as np

def rename_kicking_columns(df):
    col_counter = {}
    for col in df.columns:
        if col not in col_counter:
            col_counter[col] = 1
        else:
            col_counter[col] += 1

    new_col_names = []
    temp_col_counts = {}

    for col in df.columns:
        if col in ['FGA', 'FGM']:  # specify columns to modify
            if col not in temp_col_counts:
                temp_col_counts[col] = 1
            else:
                temp_col_counts[col] += 1
            
            # Determine naming based on position in sequence
            if temp_col_counts[col] == col_counter[col]:  # Last occurrence
                new_col_names.append(col)  # Append without number for last occurrence
            else:
                new_col_names.append(f"{col}{temp_col_counts[col]}")  # Append with number
        else:
            new_col_names.append(col)

    df.columns = new_col_names
    return df

# Apply the renaming function to the kicking_data DataFrame
kicking_data = rename_kicking_columns(kicking_data)

# Handling NaN and saving the DataFrame
kicking_data.replace('', np.nan, inplace=True)
kicking_data.dropna(subset=['FGA', 'FGM', 'FGA', 'FGM', 'FGA', 'FGM', 'FGA', 'FGM', 'FGA', 'FGM', 'FGA', 'FGM', 'Lng', 'FG%', 'XPA', 'XPM', 'XP%'], how='all', inplace=True)
kicking_data.replace(np.nan, 0, inplace=True)
kicking_data.to_csv("kicking_data.csv")

# Display the updated DataFrame and column names
print(kicking_data.head())
print(kicking_data.columns)
print(kicking_data.dtypes)  # This will show the data types of all columns




            Player   Tm Age Pos   G GS FGA1 FGM1 FGA2 FGM2  ... FGA5 FGM5 FGA  \
0   Dustin Hopkins  WAS  26   K  16  0    0    0   12   12  ...    7    3  42   
1    Caleb Sturgis  PHI  27   K  16  0    0    0   12   12  ...    6    4  41   
2       Nick Novak  HOU  35   K  16  0    1    1   12   12  ...    6    3  41   
3  Justin Tucker*+  BAL  27   K  16  0    0    0    4    4  ...   10   10  39   
4      Graham Gano  CAR  29   K  16  0    0    0    4    4  ...    6    3  38   

  FGM Lng    FG% XPA XPM     XP%  Year  
0  34  53  81.0%  39  36   92.3%  2016  
1  35  55  85.4%  31  30   96.8%  2016  
2  35  53  85.4%  25  22   88.0%  2016  
3  38  57  97.4%  27  27  100.0%  2016  
4  30  54  78.9%  34  31   91.2%  2016  

[5 rows x 24 columns]
Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'FGA1', 'FGM1', 'FGA2', 'FGM2',
       'FGA3', 'FGM3', 'FGA4', 'FGM4', 'FGA5', 'FGM5', 'FGA', 'FGM', 'Lng',
       'FG%', 'XPA', 'XPM', 'XP%', 'Year'],
      dtype='object')
Player    object
Tm   

In [74]:
import pandas as pd
columns_to_convert = ['G','XPA', 'XPM', 'FGA1', 'FGM1', 'FGA2', 'FGM2', 'FGA3', 'FGM3', 'FGA4', 'FGM4', 'FGA5', 'FGM5', 'FGA', 'FGM']
for col in columns_to_convert:
    kicking_data[col] = pd.to_numeric(kicking_data[col], errors='coerce')
# Recalculate league percentages for extra points and field goals by season
league_stats = kicking_data.groupby('Year').agg({
    'XPA': 'sum',
    'XPM': 'sum',
    'FGA1': 'sum',
    'FGM1': 'sum',
    'FGA2': 'sum',
    'FGM2': 'sum',
    'FGA3': 'sum',
    'FGM3': 'sum',
    'FGA4': 'sum',
    'FGM4': 'sum',
    'FGA5': 'sum',
    'FGM5': 'sum',
    'FGA': 'sum',
    'FGM': 'sum'
}).reset_index()

# Calculate percentages
league_stats['lg_xp_pct'] = league_stats['XPM'] / league_stats['XPA']
league_stats['lg_fg1_pct'] = league_stats['FGM1'] / league_stats['FGA1']
league_stats['lg_fg2_pct'] = league_stats['FGM2'] / league_stats['FGA2']
league_stats['lg_fg3_pct'] = league_stats['FGM3'] / league_stats['FGA3']
league_stats['lg_fg4_pct'] = league_stats['FGM4'] / league_stats['FGA4']
league_stats['lg_fg5_pct'] = league_stats['FGM5'] / league_stats['FGA5']
league_stats['lg_fg_pct'] = league_stats['FGM'] / league_stats['FGA']

# Merge league stats with the main kicking_data
kicking_data = kicking_data.merge(league_stats[['Year', 'lg_xp_pct', 'lg_fg1_pct', 'lg_fg2_pct', 'lg_fg3_pct', 'lg_fg4_pct', 'lg_fg5_pct', 'lg_fg_pct']], on='Year', how='left')

kicking_data['PAA_xp'] = np.where(kicking_data['XPA'] > 0, kicking_data['XPM'] - (kicking_data['XPA'] * kicking_data['lg_xp_pct']), 0)
kicking_data['PAA_fg1'] = np.where(kicking_data['FGA1'] > 0, 3 * (kicking_data['FGM1'] - (kicking_data['FGA1'] * kicking_data['lg_fg1_pct'])), 0)
kicking_data['PAA_fg2'] = np.where(kicking_data['FGA2'] > 0, 3 * (kicking_data['FGM2'] - (kicking_data['FGA2'] * kicking_data['lg_fg2_pct'])), 0)
kicking_data['PAA_fg3'] = np.where(kicking_data['FGA3'] > 0, 3 * (kicking_data['FGM3'] - (kicking_data['FGA3'] * kicking_data['lg_fg3_pct'])), 0)
kicking_data['PAA_fg4'] = np.where(kicking_data['FGA4'] > 0, 3 * (kicking_data['FGM4'] - (kicking_data['FGA4'] * kicking_data['lg_fg4_pct'])), 0)
kicking_data['PAA_fg5'] = np.where(kicking_data['FGA5'] > 0, 3 * (kicking_data['FGM5'] - (kicking_data['FGA5'] * kicking_data['lg_fg5_pct'])), 0)
kicking_data['PAA_fg_u'] = np.where(kicking_data['FGA'] > 0, 3 * (kicking_data['FGM'] - (kicking_data['FGA'] * kicking_data['lg_fg_pct'])), 0)

# Sum up all PAA values to get the total PAA
kicking_data['PAA_total'] = (
    kicking_data['PAA_xp'] + 
    kicking_data['PAA_fg1'] + 
    kicking_data['PAA_fg2'] + 
    kicking_data['PAA_fg3'] + 
    kicking_data['PAA_fg4'] + 
    kicking_data['PAA_fg5'] + 
    kicking_data['PAA_fg_u'])

kicking_data['avg_AV'] = (3.125 / 16) * kicking_data['G']

# Adjust average AV based on PAA_total, scaling by dividing by 5
kicking_data['raw_AV'] = kicking_data['avg_AV'] + (kicking_data['PAA_total'] / 5)

# Prorate to a full 16-game season for seasons with unusual schedules
kicking_data['AV'] = 16 * (kicking_data['raw_AV'] / kicking_data['G'])
kicking_data = kicking_data[['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'FGA1', 'FGM1', 'FGA2', 'FGM2',
       'FGA3', 'FGM3', 'FGA4', 'FGM4', 'FGA5', 'FGM5', 'FGA', 'FGM', 'Lng',
       'FG%', 'XPA', 'XPM', 'XP%', 'Year', 'AV']]
# Save the updated kicking_data DataFrame to a new CSV file
kicking_data

,Player,Tm,Age,Pos,G,GS,FGA1,FGM1,FGA2,FGM2,...,FGM5,FGA,FGM,Lng,FG%,XPA,XPM,XP%,Year,AV
0,Dustin Hopkins,WAS,26,K,16,0,0,0,12,12,...,3,42,34,53,81.0%,39,36,92.3%,2016,1.245879
1,Caleb Sturgis,PHI,27,K,16,0,0,0,12,12,...,4,41,35,55,85.4%,31,30,96.8%,2016,3.736726
2,Nick Novak,HOU,35,K,16,0,1,1,12,12,...,3,41,35,53,85.4%,25,22,88.0%,2016,3.208753
3,Justin Tucker*+,BAL,27,K,16,0,0,0,4,4,...,10,39,38,57,97.4%,27,27,100.0%,2016,10.885803
4,Graham Gano,CAR,29,K,16,0,0,0,4,4,...,3,38,30,54,78.9%,34,31,91.2%,2016,1.225988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,Michael Badgley,DET,28,K,4,0,0,0,0,0,...,0,4,4,41,100.0%,15,13,86.7%,2023,4.249459
341,Austin Seibert,NYJ,27,K,1,0,0,0,0,0,...,0,1,1,34,100.0%,1,1,100.0%,2023,5.130587
342,Dare Ogunbowale,HOU,29,RB,12,0,0,0,1,1,...,0,1,1,29,100.0%,0,0,0,2023,3.250622
343,Jamie Gillan,NYG,26,P,17,0,0,0,0,0,...,0,1,1,40,100.0%,0,0,0,2023,3.319373


In [75]:
kicking_data.columns
kicking_data.to_csv('kicking_data.csv')